# 降雨数据初步清理

### 2017-1-21日

hi,米兰哥，还不知道怎么称呼您比较好。

之前的数据我在整理的时候发现有异常，就重新做了清理。这花了一天才搞完，因为您导出的数据时间格式很不统一。现在新的图做出来了，但是新的问题出现了，比如2017年的数据就不如之前好看了。

这样导致我不知道选取哪个进行清理比较好。请您先过目。

以下是代码和图。

In [1]:
#==================
#       水数据修整
#       FanMin
#       Estabilish:20180101
#       Last:20180119
#==================
library(readxl)
library(ggplot2)
library(reshape2)

In [2]:
setwd("/Users/fanmin/Documents/GitHub/R/Water_Beijing")

In [49]:
foldernames <-paste("/Users/fanmin/Documents/GitHub/R/Water_Beijing",dir(pattern = "milan"),sep = "/")
for (yearsfolder in foldernames) {
        year <- substr(yearsfolder, 53, 59)
        print(yearsfolder)
        setwd(yearsfolder)
        filenames <- list.files(pattern = "*.csv")
        for (fns in filenames) {
                temp <-
                        read.csv(
                                fns,
                                fileEncoding = "GB2312",
                                skip = 1,
                                stringsAsFactors = F
                        )
                if (fns == list.files()[1]) {
                        temp$X. <- substr(fns, 1, 2)
                        db <- temp
                } else{
                        temp$X. <- substr(fns, 1, 2)
                        db <- rbind(db, temp)
                }
        }
        dat <- db
        colnames(dat) <-c("ID","Date","Water.Content.20m","Water.Content.40m","Water.Content.60m","Water.Content.80m","Water.Content.100m")
        if (year == "2013") {
                dat$Date <-
                        as.POSIXct(strptime(dat$Date, "%m/%d/%y %H时%M分%S秒"))
        } else{
                dat$Date <- gsub("上午", "AM ", dat$Date)
                dat$Date <- gsub("下午", "PM ", dat$Date)
                dat$Date <-
                        as.POSIXct(strptime(dat$Date, "%m/%d/%y %p %I时%M分%S秒"))
        }
        dat$Water.Content.20m <- as.numeric(dat$Water.Content.20m)
        dat$Water.Content.40m <- as.numeric(dat$Water.Content.40m)
        dat$Water.Content.60m <- as.numeric(dat$Water.Content.60m)
        dat$Water.Content.80m <- as.numeric(dat$Water.Content.80m)
        dat$Water.Content.100m <- as.numeric(dat$Water.Content.100m)
        print(paste("以下是",year, "年数据的基本情况", sep =
                            ""))
        print(summary(dat))
        cat("\n")
        dat <- melt(dat, id.vars = c("Date", "ID"))
        cat("\n")
}

[1] "/Users/fanmin/Documents/GitHub/R/Water_Beijing/"
[1] "以下是年数据的基本情况"
      ID                 Date       Water.Content.20m Water.Content.40m
 Length:42983       Min.   :NA      Min.   :-0.3977   Min.   :-0.3418  
 Class :character   1st Qu.:NA      1st Qu.: 0.0918   1st Qu.: 0.0882  
 Mode  :character   Median :NA      Median : 0.1332   Median : 0.1201  
                    Mean   :NA      Mean   : 0.1713   Mean   : 0.2260  
                    3rd Qu.:NA      3rd Qu.: 0.1724   3rd Qu.: 0.1644  
                    Max.   :NA      Max.   : 1.3338   Max.   : 2.5722  
                    NA's   :42983                                      
 Water.Content.60m Water.Content.80m Water.Content.100m
 Min.   :0.0460    Min.   :-0.1748   Min.   :0.01550   
 1st Qu.:0.1100    1st Qu.: 0.0758   1st Qu.:0.06350   
 Median :0.1434    Median : 0.1020   Median :0.07440   
 Mean   :0.2836    Mean   : 0.1144   Mean   :0.08389   
 3rd Qu.:0.1753    3rd Qu.: 0.1376   3rd Qu.:0.10780   
 Max.   :1.7899 

现在的基本情况如图所示，请您看看打算先处理哪个数据吧。

不过话说回来，这几个数据的情况，都不容乐观。

* 检查了几次，还是觉得数据有问题，可能是跟编码有问题。
* 直接索性用 python 写了合并代码。

删负值